In [1]:
from typing import List
import torch
import torch.nn as nn
from pytorch_pretrained_bert.modeling import BertModel

In [2]:
# BERT model をロード
bert_model_dir = '/mnt/larch/share/bert/Japanese_models/Wikipedia/L-12_H-768_A-12_E-30_BPE'
bert_model = BertModel.from_pretrained(bert_model_dir)

In [3]:
# config を確認
bert_model.config

{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 32006
}

In [4]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
# BPE で形態素をサブワードにトークナイズするためのオブジェクト
tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case=False) # 濁点対策
(list(tokenizer.vocab)[:10],
list(tokenizer.vocab)[3000:3010])

(['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', 'の', '、', '。', 'に', 'は'],
 ['加', '世帯', 'テスト', 'ホー', '羽', '##公', '##西', '出た', '##型', '##ラス'])

In [5]:
input_sentences: List[str] = [
    '７ 　 自然 と 人間 野山 や 川 ， 海 など ， さまざまな 場所 に 数 多く の 生物 が 生活 して い ます 。',
    '寂聴 　 ’ ０４ 　 みちのく 青空 説法 夏 の 法話 ― 出家 した 翌年 の 桜 は びっくり する ほど きれでした',
    '屋久島 の 酸性 雨 被害 など 報告 大阪 で 危機 管理 講座 学校 法人 加計 学園 、'
]

In [6]:
input_tokens: List[List[str]] = []
for sentence in input_sentences:
    input_tokens.append(['[CLS]'] + tokenizer.tokenize(sentence) + ['[SEP]'])
[' '.join(tokens) for tokens in input_tokens]  # トークナイズされた文

['[CLS] ７ 自然 と 人間 野 ##山 や 川 ， 海 など ， さまざまな 場所 に 数 多く の 生物 が 生活 して い ます 。 [SEP]',
 '[CLS] 寂 ##聴 ’ ０４ みちの ##く 青空 説 ##法 夏 の 法 ##話 ― 出家 した 翌年 の 桜 は び ##っくり する ほど きれ ##で ##した [SEP]',
 '[CLS] 屋 ##久 ##島 の 酸性 雨 被害 など 報告 大阪 で 危機 管理 講座 学校 法人 加 ##計 学園 、 [SEP]']

In [7]:
input_ids: List[List[int]] = []
for tokens in input_tokens:
    input_ids.append(tokenizer.convert_tokens_to_ids(tokens))
[' '.join(str(id_) for id_ in ids) for ids in input_ids]  # トークナイズ&ID化された文

['2 75 1140 12 652 1981 444 34 246 176 573 42 176 2884 463 8 145 135 5 1323 11 580 19 142 1953 7 3',
 '2 17394 11165 699 10632 26592 712 24852 791 1643 835 5 202 5143 6086 9746 20 1044 5 2847 9 5305 19537 22 500 7860 429 1033 3',
 '2 1552 2191 760 5 17609 2899 1412 42 1036 340 13 2783 567 7015 172 907 3001 12845 1769 6 3']

In [8]:
# segment_id(token_type_ids) と input_mask(attention_mask) を作成
max_seq_len: int = max(len(ids) for ids in input_ids)  # トークン列の最大長
segment_ids: List[List[int]] = []
input_mask: List[List[int]] = []
for idx, ids in enumerate(input_ids):
    seq_len = len(ids)
    pad: List[int] = [0] * (max_seq_len - seq_len)  # パディング
    input_ids[idx] += pad
    segment_ids.append([0] * max_seq_len) # 文対を扱うタスクではないので全てゼロ
    input_mask.append([1] * seq_len + pad)

In [9]:
# 最初の文に対して input_ids, segment_ids, input_mask を表示
(' '.join(str(x) for x in input_ids[0]),
' '.join(str(x) for x in segment_ids[0]),
' '.join(str(x) for x in input_mask[0]))

('2 75 1140 12 652 1981 444 34 246 176 573 42 176 2884 463 8 145 135 5 1323 11 580 19 142 1953 7 3 0 0',
 '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0',
 '1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0')

In [10]:
# パディングできたので torch.Tensor に変換
input_ids = torch.tensor(input_ids)      # (3, 27)
segment_ids = torch.tensor(segment_ids)  # (3, 27)
input_mask = torch.tensor(input_mask)    # (3, 27)

In [11]:
# GPUに送る
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # 各自、空いている gpu_id に書き換え
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
input_mask = input_mask.to(device)

In [12]:
# forward
encoded_layers, pooled_output = bert_model(input_ids, 
                                           token_type_ids=segment_ids,
                                           attention_mask=input_mask,
                                           output_all_encoded_layers=False)
(encoded_layers.size(), pooled_output.size())

(torch.Size([3, 29, 768]), torch.Size([3, 768]))

In [13]:
additional_layer = nn.Linear(768, 4).to(device)  # 4クラス分類の時
output = additional_layer(pooled_output)
output.size()
# このあと、cross entropy loss をとったり、back propagation したり

torch.Size([3, 4])